In [ ]:

print("Evaluating Model on test data...")
model.evaluate([W_test], Y_test, verbose=1)

In [ ]:
print("Evaluating Model on train data...")
model.evaluate([W_train], Y_train, verbose=1)

In [ ]:
import pandas as pd
import pickle
import torch
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm, tqdm_notebook
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import gensim
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from collections import defaultdict
from nltk.data import load
from tqdm import tqdm
import time
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
import os
import nltk
import spacy
import networkx as nx
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import tensorflow.keras
from tensorflow.keras.callbacks import ModelCheckpoint
!pip install tensorboardcolab
from tensorboardcolab import *
import tensorflow_hub as hub
import tensorflow as tf

from keras import initializers, regularizers, constraints

seed = 0

torch.manual_seed(seed)
np.random.seed(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
embedding_size=100
vocab_size= np.shape(embedding_matrix)[0]
embedding_dim=64
filter_sizes=[3,4,5]
reshape = Reshape((105,100,1))(embedding)
reshape_sdp = Reshape((105,100,1))(short_embedding)
conv_1 = Conv2D(64, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

drop1 = Dropout(0.5)(conv_1)
max_pool1 = MaxPool2D(pool_size=(105 - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(drop1)  
flat1 = Flatten()(max_pool1)


conv_2 = Conv2D(64, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

drop2 = Dropout(0.5)(conv_2)
max_pool2 = MaxPool2D(pool_size=(105 - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(drop2)  
flat2 = Flatten()(max_pool2)

print(flat2)
print(np.shape(embedding_matrix_sdp)[0])

s_conv_1 = Conv2D(64, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape_sdp)

s_drop1 = Dropout(0.3)(s_conv_1)
s_max_pool1 = MaxPool2D(pool_size=(105 - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(s_drop1)  
s_flat1 = Flatten()(s_max_pool1)


s_conv_2 = Conv2D(64, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape_sdp)

s_drop2 = Dropout(0.3)(s_conv_2)
s_max_pool2 = MaxPool2D(pool_size=(105 - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(s_drop2)  
s_flat2 = Flatten()(s_max_pool2)


print('khaaaaaaaar')
concatenated_tensor = Concatenate(axis=1)([max_pool1, max_pool2])
        
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(0.2)(flatten)
output = Dense(units=5, activation='softmax')(dropout)

model = Model(inputs=[inputs], outputs=output)

adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='mse', metrics=['accuracy', precision_m, recall_m, f1_m])
model.summary()


yt=Y_train.copy()
yt=np.argmax(Y_train,axis=-1)
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(yt),
                                                 yt)
print(class_weights)

model.fit([W_train], Y_train,
                 epochs = 20, shuffle=True, batch_size=200, verbose = 1
                 )


In [ ]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
import tensorflow
K.clear_session()
embedding_layer_sdp = tensorflow.keras.layers.Embedding(np.shape(embedding_matrix_sdp)[0],
                            100,
                            weights=[embedding_matrix_sdp],
                            input_length=105,
                            trainable=False)
s_inputs = tensorflow.keras.layers.Input(shape=(105,), dtype='int32')
short_embedding = embedding_layer_sdp(s_inputs)

In [ ]:
embedding_layer = tensorflow.keras.layers.Embedding(np.shape(embedding_matrix)[0],
                            100,
                            weights=[embedding_matrix],
                            input_length=105,
                            trainable=False)
inputs = tensorflow.keras.layers.Input(shape=(105,), dtype='int32')
embedding = embedding_layer(inputs)
input2=Input(shape=(sentMax,),name='d1')
input3=Input(shape=(sentMax,),name='d2')
input4=Input(shape=(sentMax,),name='pos')
x2=Embedding(d1_dict_size, 10,trainable=True,input_length=sentMax)(input2)
x3=Embedding(d2_dict_size, 10,trainable=True,input_length=sentMax)(input3)
x4=Embedding(pos_dict_size, 4,trainable=True,input_length=sentMax)(input4)

In [ ]:
embedding_matrix_sdp = np.zeros((len(word_index_sdp) + 1, 100))
for word, i in word_index_sdp.items():
    embedding_vector_sdp = embeddings_index.get(word)
    if embedding_vector_sdp is not None:
        embedding_matrix_sdp[i] = embedding_vector_sdp

In [ ]:
train_data_sdp=[]
test_data_sdp=[]
train_label_sdp=[]
test_label_sdp=[]
texts_train_sdp = []
texts_test_sdp = []
labels_sdp = []
labels_index = {'false':0, 'advise': 1, 'mechanism': 2, 'effect': 3, 'int': 4}
textss_sdp=[]

for i in range(len(tr_sdp)):
    sentence=tr_sdp[i]
    textss_sdp.append(sentence)

    

for i in range(len(te_sdp)):
    sentence=te_sdp[i]
    textss_sdp.append(sentence)

tokenizer  = Tokenizer(num_words = 10000)
tokenizer.fit_on_texts(textss_sdp)
sequences_sdp =  tokenizer.texts_to_sequences(textss_sdp)

word_index_sdp = tokenizer.word_index

In [ ]:
tr_sdp=[]
for i in train_sdp:
  tr_sdp.append(' '.join(word for word in i))
print(tr_sdp[0])

te_sdp=[]
for i in test_sdp:
  te_sdp.append(' '.join(word for word in i))
print(te_sdp[0])

In [ ]:
with open("test_sdp.txt", "wb") as fp:   #Pickling
  pickle.dump(Te_sent_contents_SDP, fp)

with open("test_sdp.txt", "rb") as fp:   # Unpickling
  test_sdp = pickle.load(fp)

print((test_sdp[10]))
print(len(test_sdp))

In [ ]:
with open("train_sdp.txt", "wb") as fp:   #Pickling
  pickle.dump(Tr_sent_contents_SDP, fp)

with open("train_sdp.txt", "rb") as fp:   # Unpickling
  train_sdp = pickle.load(fp)

print(train_sdp[0])
print(len(train_sdp))


In [ ]:
with open("train_sdp.txt", "wb") as fp:   #Pickling
  pickle.dump(Tr_sent_contents_SDP, fp)

with open("train_sdp.txt", "rb") as fp:   # Unpickling
  train_sdp = pickle.load(fp)

print(train_sdp[0])
print(len(train_sdp))


In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
train_data=[]
test_data=[]
train_label=[]
test_label=[]
texts_train = []
texts_test = []
labels = []
labels_index = {'false':0, 'advise': 1, 'mechanism': 2, 'effect': 3, 'int': 4}
textss=[]

for i in range(len(Tr_sent_contents)):
    sentence=remove_punctuation(Tr_sent_contents[i])
    textss.append(sentence)
    train_label.append(label_dict[Tr_sent_lables[i]])
    labels.append(labels_index[Tr_sent_lables[i]])

for i in range(len(Te_sent_contents)):
    sentence=remove_punctuation(Te_sent_contents[i])
    textss.append(sentence)
    test_label.append(label_dict[Te_sent_lables[i]])  
    labels.append(labels_index[Te_sent_lables[i]])

tokenizer  = Tokenizer(num_words = 10000)
tokenizer.fit_on_texts(textss)
sequences =  tokenizer.texts_to_sequences(textss)

word_index = tokenizer.word_index


In [ ]:
import os
import sys
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Activation, Conv2D, Input, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from tensorflow.keras.layers import MaxPool1D
from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam
from sklearn.utils import shuffle
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Embedding
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

In [ ]:
import os
embeddings_index = {}
base_path = ""
filename = "glove.6B.100d.txt"
path_to_file = os.path.join(base_path, filename)
f = open(path_to_file , 'r', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))


def remove_punctuation(string): 
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 
    return string

In [ ]:
sdp_train=mapWordToId(Tr_sent_contents_SDP, sdp_dict)
sdp_test=mapWordToId(Te_sent_contents_SDP, sdp_dict)
print(W_train)
sdp_train,W_train, d1_train, d2_train,T_train, pos_train,Tr_sent_contents_SDP_expand,sdp_test,W_test, d1_test, d2_test,T_test,pos_test,Te_sent_contents_SDP_expand\
=paddData([sdp_train,W_train, d1_train, d2_train,T_train,pos_train,Tr_sent_contents_SDP_expand,sdp_test, W_test, d1_test, d2_test,T_test,pos_test,Te_sent_contents_SDP_expand],
          sentMax)

#print(sdp_train)

In [ ]:


def findSentLengths(tr_te_list):
  lis = []
  for lists in tr_te_list:
    lis.append([len(l) for l in lists])
  print(lis)
  return lis

print(Tr_word_list[1])
train_sent_lengths,test_sent_lengths = findSentLengths([Tr_word_list,Te_word_list])
print(train_sent_lengths)
sentMax = max(train_sent_lengths  + test_sent_lengths)
print ("max sent length", sentMax)
sentMax=105

max(test_sent_lengths)

train_sent_lengths = np.array(train_sent_lengths, dtype='int32')
test_sent_lengths = np.array(test_sent_lengths, dtype='int32')


label_dict = {'false':0, 'advise': 1, 'mechanism': 2, 'effect': 3, 'int': 4}


def makeDistanceList(lista):
  sent_list = sum(lista, [])
  wf = {}
  for sent in sent_list:
    for w in sent:
      if w in wf:
        wf[w] += 1
      else:
        wf[w] = 0
   
  wl = []	
  i = 1
  for w,f in wf.items():
    wl.append(w)
  print(wl)
  return wl

def makeWordList(lista):  
  sent_list = sum(lista, [])
  print(len(sent_list))#sum train and test sentences.
  wf = {}
  for sent in sent_list:
    for w in sent:
      if w in wf:
        wf[w] += 1
      else:
        wf[w] = 0

  wl = []	
  i = 1

  wl.append('<pad>')
  wl.append('<unkown>')
  for w,f in wf.items():
    wl.append(w)
  print(wf)
  print(wl)
  return wl


word_dict = makeWordList([Tr_word_list, Te_word_list])
d1_dict = makeDistanceList([Tr_d1_list,  Te_d1_list,])
print(d1_dict)
d2_dict = makeDistanceList([Tr_d2_list,  Te_d2_list])
pos_dict=makeDistanceList([Tr_word_pos,Te_word_pos])
type_dict = makeWordList([Tr_type_list, Te_type_list])
sdp_dict=makeWordList([Tr_sent_contents_SDP, Te_sent_contents_SDP]) 
print ("word dictonary length", len(word_dict))





def mapLabelToId(sent_lables, label_dict):
  if len(label_dict) > 2:
    return [label_dict[label] for label in sent_lables]
  else:
    return [int (label != 'false') for label in sent_lables]

def mapWordToId(sent_contents, word_list):
  T = []
  for sent in sent_contents:
    t = []
    for w in sent:
      t.append(word_list.index(w))
    T.append(t)
  return T

# Mapping Train
W_train =   mapWordToId(Tr_word_list, word_dict)
d1_train = mapWordToId(Tr_d1_list, d1_dict)
d2_train = mapWordToId(Tr_d2_list, d2_dict)
pos_train=mapWordToId(Tr_word_pos, pos_dict)
T_train = mapWordToId(Tr_type_list,type_dict)



#One Hot Encoding
Y_t = mapLabelToId(Tr_sent_lables, label_dict)
Y_train = np.zeros((len(Y_t), len(label_dict)))
for i in range(len(Y_t)):
	Y_train[i][Y_t[i]] = 1.0


!# Mapping Test
W_test =   mapWordToId(Te_word_list, word_dict)
d1_test = mapWordToId(Te_d1_list, d1_dict)
d2_test = mapWordToId(Te_d2_list, d2_dict)
pos_test=mapWordToId(Te_word_pos, pos_dict)
T_test = mapWordToId(Te_type_list,type_dict)

Y_t = mapLabelToId(Te_sent_lables, label_dict)
Y_test = np.zeros((len(Y_t), len(label_dict)))
for i in range(len(Y_t)):
	Y_test[i][Y_t[i]] = 1.0

da=np.zeros(sentMax)
db=np.zeros(sentMax)
da[0]=4
db[0]=8
da=np.reshape(da,(1,sentMax))
db=np.reshape(db,(1,sentMax))

from tensorflow.keras.preprocessing import sequence
def paddData(listL, maxl): #W_batch, d1_tatch, d2_batch, t_batch)
  rlist = []
  import tensorflow.keras
  for mat in listL:
    mat_n=tensorflow.keras.preprocessing.sequence.pad_sequences(mat, maxlen=maxl, dtype='int32', padding='post', truncating='post', value=0.0)
    rlist.append(np.array(mat_n))
  return rlist

#padding
W_train, d1_train, d2_train,T_train, pos_train,Tr_sent_contents_SDP_expand,W_test, d1_test, d2_test,T_test,pos_test,Te_sent_contents_SDP_expand\
=paddData([W_train, d1_train, d2_train,T_train,pos_train,Tr_sent_contents_SDP_expand,W_test, d1_test, d2_test,T_test,pos_test,Te_sent_contents_SDP_expand],
          sentMax) 
            
print ("train", len(W_train))
print ("test", len(W_test))



d1_dict_size = len(d1_dict)
d2_dict_size = len(d2_dict)
pos_dict_size = len(pos_dict)
print ("d1_dict_size", d1_dict_size)
print ("d2_dict_size", d2_dict_size)
print ("pos_dict_size", pos_dict)

import tensorflow.keras.backend as K

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [ ]:
def dataRead(fname):
    print( "Input File Reading")
    fp = open(fname, 'r')
    samples = fp.read().strip().split('\n\n')
    sent_lengths   = []		#1-d array
    sent_contents  = []		#2-d array [[w1,w2,....] ...]
    sent_lables    = []		#1-d array
    entity1_list   = []		#2-d array [[e1,e1_t] [e1,e1_t]...]
    entity2_list   = []		#2-d array [[e1,e1_t] [e1,e1_t]...]
    for sample in samples:
      sent, entities, relation = sample.strip().split('\n')

      e1, e1_t, e2, e2_t = entities.split('\t') 
      sent_contents.append(sent.lower())
      entity1_list.append([e1, e1_t])
      entity2_list.append([e2, e2_t])
      sent_lables.append(relation)

    return sent_contents, entity1_list, entity2_list, sent_lables

ftrain = r'all_data_train_data.txt'
ftest = r'all_data_test_data.txt'
Tr_sent_contents, Tr_entity1_list, Tr_entity2_list, Tr_sent_lables = dataRead(ftrain)
print(Tr_sent_contents[0])
Te_sent_contents, Te_entity1_list, Te_entity2_list, Te_sent_lables = dataRead(ftest)

train=pd.DataFrame({'sents':Tr_sent_contents})
freq = pd.Series(' '.join(train['sents']).split()).value_counts()[-600:]
# freq
freq = list(freq.index)
train['sents'] = train['sents'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train['sents'].head()



test=pd.DataFrame({'sents':Te_sent_contents})
# 

freq = pd.Series(' '.join(test['sents']).split()).value_counts()[-300:]
# freq
freq = list(freq.index)
test['sents'] = test['sents'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
test['sents'].head()


Tr_sent_contents=list(train['sents'])
print(len(Tr_sent_contents))
Te_sent_contents=list(test['sents'])

def shortest_dependency_path(sents, e1=None, e2=None):
    temp=[]
    for s in sents:
      
      doc = nlp(s)
      edges = []
      for token in doc:
          for child in token.children:
              edges.append(('{0}'.format(token),
                            '{0}'.format(child)))
      graph = nx.Graph(edges)
      try:
          shortest_path = nx.shortest_path(graph, source=e1, target=e2)
      except:
          shortest_path = shortest_path = [e1,e2]
      temp.append(shortest_path)
    return temp

Tr_sent_contents_SDP=shortest_dependency_path(Tr_sent_contents, e1='druga', e2='drugb')
print(Tr_sent_contents_SDP[0])
Te_sent_contents_SDP=shortest_dependency_path(Te_sent_contents, e1='druga', e2='drugb')

print(Tr_sent_contents_SDP[1])

import re

#Replace all white-space characters with the digit "9":

txt = "The rain in Spain"
x = re.sub("\s", "9", txt)
print(x)

def preProcess(sent):
	sent = sent.lower()
	sent = sent.replace('/',' ')
	sent = sent.replace('.','')
	#print(sent)
	sent = tokenizer.tokenize(sent)
	sent = ' '.join(sent)
	sent = re.sub('\d', 'dg',sent)
	return sent

def makeFeatures(sent_list, entity1_list, entity2_list):
  print ('Making Features')
  word_list = []
  d1_list = []
  d2_list = []
  type_list = []
  for sent, ent1, ent2 in zip(sent_list, entity1_list, entity2_list):
    sent = preProcess(sent)
    sent_list1 = sent.split()
    #print(sent_list1)
    entity1 = preProcess(ent1[0]).split()
    entity2 = preProcess(ent2[0]).split()
    #print('entity1:' + entity1[0])
    #print('entity2:' + entity2[0])
    s1 = sent_list1.index('druga')
    s2 = sent_list1.index('drugb') 
    #print(s1)
    #print(s2)
		# distance1 feature	
    d1 = []
    for i in range(len(sent_list1)):
        if i < s1 :
           d1.append(str(i - s1))
        elif i > s1 :
           d1.append(str(i - s1 ))
        else:
           d1.append('0')
		#distance2 feature		
    d2 = []
    for i in range(len(sent_list1)):
        if i < s2:
           d2.append(str(i - s2))
        elif i > s2:
           d2.append(str(i - s2))
        else:
           d2.append('0')
    t = []
    for i in range(len(sent_list1)):
      t.append('Out')
    t[s1] = ent1[1]		
    t[s2] = ent2[1]

    word_list.append(sent_list1)
    d1_list.append(d1)
    d2_list.append(d2)
    type_list.append(t) 

  #print(sent_list1)
  #print(d1)
  #print(d2)
  return word_list, d1_list, d2_list,type_list

Tr_word_list, Tr_d1_list, Tr_d2_list, Tr_type_list = makeFeatures(Tr_sent_contents, Tr_entity1_list, Tr_entity2_list)
print(Tr_word_list[0])                                                                    
                                                                            

Tr_word_pos_t=[]
for i in Tr_word_list:
  #print(i)
    #print(j)
    
  Tr_word_pos_t.append(nltk.pos_tag(i))

print(Tr_word_pos_t[0])

Tr_word_pos=[]
for i in range(np.shape(Tr_word_pos_t)[0]):
  temp=[]
  for j in range(np.shape(Tr_word_pos_t[i])[0]):
    temp.append(Tr_word_pos_t[i][j][1])
  Tr_word_pos.append(temp)

print(Tr_word_pos[0])

Te_word_list, Te_d1_list, Te_d2_list, Te_type_list = makeFeatures(Te_sent_contents, Te_entity1_list, Te_entity2_list)                                
print(Te_word_list)
Te_word_pos_t=[]
for i in Te_word_list:
  Te_word_pos_t.append(nltk.pos_tag(i))

Te_word_pos=[]
for i in range(np.shape(Te_word_pos_t)[0]):
  temp=[]
  for j in range(np.shape(Te_word_pos_t[i])[0]):
    temp.append(Te_word_pos_t[i][j][1])
  Te_word_pos.append(temp)

def makeWordList(lista):  
  sent_list = sum(lista, [])
  print(len(sent_list))#sum train and test sentences.
  wf = {}
  for sent in sent_list:
    for w in sent:
      if w in wf:
        wf[w] += 1
      else:
        wf[w] = 0

  wl = []	
  i = 1

  wl.append('<pad>')
  wl.append('<unkown>')
  for w,f in wf.items():
    wl.append(w)
  print(wf)
  print(wl)
  return wl

word_dict = makeWordList([Tr_word_list, Te_word_list])
print ("word dictonary length", len(word_dict))

Tr_sent_contents_SDP_expand=[]

for i in range(np.shape(Tr_word_list)[0]):
  temp=[]
  for j in range(np.shape(Tr_word_list[i])[0]):
    
    if Tr_word_list[i][j] in Tr_sent_contents_SDP[i]:
      temp.append(1)
    else:
      temp.append(0)
  
  Tr_sent_contents_SDP_expand.append(temp)    
  
print(Tr_sent_contents_SDP_expand[0])

Te_sent_contents_SDP_expand=[]

for i in range(np.shape(Te_word_list)[0]):
  temp=[]
  for j in range(np.shape(Te_word_list[i])[0]):
    
    if Te_word_list[i][j] in Te_sent_contents_SDP[i]:
      temp.append(1)
    else:
      temp.append(0)
  
  Te_sent_contents_SDP_expand.append(temp)
print( Te_sent_contents_SDP_expand[0])

In [ ]:
!apt-get -qq install -y graphviz && pip install -q pydot
import pydot
import nltk
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()
import operator
def frequentWord(sents):
  wf = {}
  for s in sents:
    for w in s:
      if w in wf:
        wf[w]+=1
      else:
        wf[w]=0

  sorted_x = sorted(wf.items(), key=operator.itemgetter(1),reverse=True)
  return sorted_x

import scispacy
import spacy

#nlp = en_core_sci_sm.load()

import networkx as nx
import spacy
from nltk import Tree
nlp = spacy.load('en_core_sci_lg')

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

import numpy as np
import pandas  as pd
import sklearn as sk
import random
import csv
import re
import collections
import pickle
import sys
!pip install nltk
import nltk
nltk.download('averaged_perceptron_tagger')
wv_embSize = 100
d1_emb_size=10
d2_emb_size=10
type_emb_size=10
numfilter = 200
LSTM_unit = 200
num_epochs = 18
batch_size=200
reg_para = 0.001
drop_out = 1.0

In [ ]:
!pip uninstall h5py
!pip install h5py==2.10.0

In [ ]:
!pip install spacy
!pip install scispacy
!pip3 install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_scibert-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_lg-0.4.0.tar.gz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/
!ls
